In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_649097/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.4)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_45_9,0.999721,0.821340,0.809846,0.999600,0.001259,1.194699,0.680172,0.001568,0.085964,0.035476,1.002315,0.036093,143.355579,278.395513,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_45_8,0.999719,0.821404,0.809962,0.999614,0.001265,1.194271,0.679755,0.001512,0.086770,0.035562,1.002326,0.036181,143.345904,278.385838,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_45_7,0.999717,0.821439,0.810013,0.999623,0.001275,1.194035,0.679573,0.001475,0.087331,0.035707,1.002345,0.036328,143.329631,278.369565,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_45_6,0.999715,0.821490,0.810109,0.999635,0.001284,1.193698,0.679228,0.001429,0.087905,0.035833,1.002362,0.036456,143.315543,278.355477,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_45_5,0.999712,0.821510,0.810149,0.999643,0.001297,1.193564,0.679085,0.001399,0.088468,0.036018,1.002386,0.036644,143.295000,278.334934,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,model_1_0_4,0.958452,0.753013,0.792084,0.916095,0.187157,1.651601,0.743705,0.387564,0.384121,0.432616,1.344255,0.440141,133.351616,268.391550,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
493,model_1_0_3,0.957802,0.752187,0.791692,0.915318,0.190086,1.657130,0.745106,0.391151,0.386319,0.435989,1.349643,0.443572,133.320557,268.360490,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
494,model_1_0_2,0.957138,0.751347,0.791289,0.914526,0.193076,1.662741,0.746546,0.394811,0.388550,0.439404,1.355142,0.447046,133.289346,268.329280,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
495,model_1_0_1,0.956460,0.750495,0.790876,0.913715,0.196129,1.668441,0.748025,0.398556,0.390799,0.442864,1.360758,0.450567,133.257966,268.297900,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
